Imports

In [1]:
import org.apache.spark.sql.types.{StructField, StructType, StringType, IntegerType}
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.SparkContext
import org.apache.spark.mllib.random.RandomRDDs._
import org.apache.spark.rdd
import org.apache.spark.sql.DataFrameReader
import org.apache.spark.mllib.rdd.RDDFunctions._

1) Write a function that will put N doubles into a file. The doubles need to be normally distributed with mean 0 and standard deviation 1. The function should have two arguments: N and the full name of the file (ie includes path to file location).

In [2]:
def nDoublesNormalDistribution(N:Int, path: String) : Unit = {
    val result = normalRDD(sc, N)
    result.saveAsTextFile(path)
}

nDoublesNormalDistribution(100,"output1")


Create a file with 50,000 doubles using the function from problem 1. This file will be used for the next several problems. It is best if you put the file in the current directory to avoid paths that do not exist on other machines.

In [3]:
def nDoublesNormalDistribution(N:Int, path: String) : Unit = {
    val result = normalRDD(sc, N)
    result.saveAsTextFile(path)
}

nDoublesNormalDistribution(50000,"output.txt")


2) Read the file created in #2 into an RDD and compute the mean and standard deviation of the doubles in the file. Work on the RDD, that is do not convert the RDD to a DataFrame or Dataset.. You are to use Spark code to compute the values as we want this to run on a cluster using multiple machines. So the pure Scala code you used in assignment will not work.

In [4]:
val outputFileData = sc.textFile("output.txt").map(x => x.toDouble)
println("Mean :" + outputFileData.mean)
print("Standard Deviation :" + outputFileData.stdev)

Mean :-0.0010871448803535851
Standard Deviation :1.0006187579378962

3) Repeat #3 but using a DataFrame instead of RDD. Here work on the DataFrame not an RDD.

In [5]:
val dataFrame = spark.read.text("output.txt")
val meanAndStandradDeviation = dataFrame.selectExpr("mean(value) as `Mean`","stddev(value) as `Standard Deviation`")
meanAndStandradDeviation.show()

+--------------------+------------------+
|                Mean|Standard Deviation|
+--------------------+------------------+
|-0.00108714488035...|1.0006287642755711|
+--------------------+------------------+



4) Using a DataFrame create a random sample of about 100 elements of the file created in #2 and compute the mean of the sample. 

In [7]:
val dataFrameSample = spark.read.text("output.txt")
val sampledData = dataFrameSample.sample(false,0.003)
val sampledDataHundered = sampledData.limit(100)
val meanSampledData = sampledDataHundered.selectExpr("mean(value) as `Sample Mean`","count(value) as `Number of Elements`")
meanSampledData.show()

+-------------------+------------------+
|        Sample Mean|Number of Elements|
+-------------------+------------------+
|0.02866426042080432|               100|
+-------------------+------------------+



5) Create a file of 100 normally distributed doubles. Read the doubles from the file into an RDD. Using the RDD create a sliding window of size 20 and compute the mean of each window. 

In [9]:
def hunderedDoublesNormalDistribution(N:Int, path: String) : Unit = {
    val result = normalRDD(sc, N)
    result.saveAsTextFile(path)
}

hunderedDoublesNormalDistribution(100,"HunderedDoubles.txt")

In [10]:
val HunderedDoublesData = sc.textFile("HunderedDoubles.txt").map(x => x.toDouble)
val slidingMeans = HunderedDoublesData.sliding(20).map(x => (x.sum/x.size))
println ("The means of the windows of size 20: ")
slidingMeans.collect().foreach(println)
println ("The number of windows: ")
slidingMeans.collect().size

The means of the windows of size 20: 
0.1835441334650994
0.28727539528616813
0.19979783925901715
0.20210602542027423
0.11973748267104237
-0.0381558335979186
0.015940549390319286
0.06056421060304512
0.05857339655049243
0.0421041007779791
0.04124518473699818
-0.07188843413672932
-0.08217699652981544
-0.03313465064788966
-0.038366522417607206
0.04147658787665366
0.0767590709241835
0.06431586377173801
0.01924737351638612
-0.07695497392378084
-0.1299832974259568
-0.27608362943444875
-0.11700843153107801
-0.16247904309039463
-0.0771826080789622
-0.03329760090075074
-0.016771004609188244
0.06577533390309094
0.06361799408135461
0.0886192657368158
0.1394550493320574
0.17246661920161815
0.16409858296646646
0.166789393852598
0.10792332080083225
0.03130269799415454
0.11315986110548366
0.20655326492698914
0.17997412953179698
0.18288428542182736
0.17812682711015398
0.2780840793195744
0.1342364084257318
0.14072284425141146
0.06207186041902764
0.06302053268590116
0.13748253046464315
0.0622682388941962

81

6) The file “multiple-sites.tsv” contains two columns: site and dwell-time. Using Spark compute the average dwell time for each site.

In [ ]:
val reader = spark.read

val manualSchema = new StructType(Array(
new StructField("site", IntegerType, false),
new StructField("dwell-time", IntegerType, false)
))

val multipleSites = reader.format("csv").option("header",true).
option("delimiter", "\t").schema(manualSchema).load("multiple-sites.tsv")

val groupedMultipleSites = multipleSites.groupBy("site")
groupedMultipleSites.mean("dwell-time").orderBy(("site")).show

7) The file “multiple-sites.tsv” contains two columns: date and dwell-time. Using Spark compute the following:
1. The average dwell time each hour
2. The average dwell time per day of week
3. The average dwell time on week-days (Monday - Friday)
4. Average dwell time on the weekend.

In [2]:
val reader = spark.read

val multipleDateDwellTimes = reader.format("csv").option("header",true).option("inferSchema",true).
option("sep", "\t").load("dwell-times.tsv")

val multipleDateSepearted = multipleDateDwellTimes.withColumnRenamed("date", "TimeStamp").
withColumn("Date", to_date(col("TimeStamp"))).
withColumn("Hour", hour(col("TimeStamp"))).
withColumn("Month", date_format(col("Date"),"MMMM")).
withColumn("Day",from_unixtime(unix_timestamp(col("Date")), "EEEEE"))


val averageDwellTimePerHour = multipleDateSepearted.groupBy("Hour")
val averageDwellTimePerHourValue = averageDwellTimePerHour.mean("dwell-time").orderBy("Hour").
withColumnRenamed("avg(dwell-time)","Average Dwell Time each Hour")
averageDwellTimePerHourValue.show(25)

val averageDwellPerDay = multipleDateSepearted.groupBy("Day")
val weekAverageDays = averageDwellPerDay.mean("dwell-time").
withColumnRenamed("avg(dwell-time)","Average Dwell Time per day of the week")
weekAverageDays.show()

val weekDaysAverage = weekAverageDays.
filter(not(weekAverageDays("Day") === "Sunday")).filter(not(weekAverageDays("Day") === "Saturday")).
select(mean("Average Dwell Time per day of the week")).
withColumnRenamed("avg(Average Dwell Time per day of the week)","Average Dwell Time on Weekdays")
weekDaysAverage.show()

val weekendsAverage = weekAverageDays.
filter(weekAverageDays("Day") === "Sunday" || weekAverageDays("Day") === "Saturday").
select(mean("Average Dwell Time per day of the week")).
withColumnRenamed("avg(Average Dwell Time per day of the week)","Average Dwell Time on Weekend")
weekendsAverage.show()

+----+----------------------------+                                             
|Hour|Average Dwell Time each Hour|
+----+----------------------------+
|   0|             94.708670095518|
|   1|           92.20954287620954|
|   2|           96.85937970490816|
|   3|           92.57110924839341|
|   4|           91.33086825527253|
|   5|           96.14621040723982|
|   6|           92.17173727608757|
|   7|           94.68557487038731|
|   8|           92.35551839464883|
|   9|           90.15113156885309|
|  10|           92.52882433045846|
|  11|           92.37820848611838|
|  12|           91.21507890122736|
|  13|           92.82834185536888|
|  14|           95.47408666100254|
|  15|           90.56982193064667|
|  16|            93.1575817641229|
|  17|           91.66231155778894|
|  18|           93.13833634719711|
|  19|            88.4258114374034|
|  20|           95.00888324873097|
|  21|           91.80379403794038|
|  22|           97.38431752178121|
|  23|           93

8) Do the average dwell times computed in #7 indicate any difference in users behavior?

- The user spend the maximum amount of time on the website at the 22nd hour of the day (dwell-time: 97.38431752178121) and the least amount of time on the website in the 19th hour of the day (dwell-time:  88.4258114374034)

- It can also be inferred that the users spends less time on the website in the weekdays(90.4719533927357) and more time on it on the weekends (111.68629346933187|)